In [1]:
import pandas as pd 
import numpy as np
import os

In [32]:
pd.set_option('display.max_columns', None)

# Games base  

In [2]:
games = pd.read_csv(os.getcwd() + "/artifacts/fetched_data/get_game_list.csv")
games.head(5)

,GW,id,home,team_h,away,team_a,finished,kickoff,team_h_score,team_a_score,season_start_year
0,1,1,CRY,7,ARS,1,True,2022-08-05T19:00:00Z,0.0,2.0,22
1,1,4,FUL,9,LIV,12,True,2022-08-06T11:30:00Z,2.0,2.0,22
2,1,2,BOU,3,AVL,2,True,2022-08-06T14:00:00Z,2.0,0.0,22
3,1,5,LEE,11,WOL,20,True,2022-08-06T14:00:00Z,2.0,1.0,22
4,1,7,NEW,15,NFO,16,True,2022-08-06T14:00:00Z,2.0,0.0,22


In [3]:
# an ID can get a new team for another season, so this could be used.
#team_id = pd.concat([games[['season_start_year', 'team_h', 'home']].rename(columns = {'home':'team', 'team_h':'team_id_season'}),
#                     games[['season_start_year', 'team_a', 'away']].rename(columns = {'away':'team', 'team_a':'team_id_season'})
#            ]).drop_duplicates()

In [4]:
games['kickoff'] = pd.to_datetime(games['kickoff'])
games['kickoff_year'] = games['kickoff'].dt.year
games['kickoff_month'] = games['kickoff'].dt.month
games['rounds_left'] = 38-games['GW']
games['label_1'] = np.where(games['team_h_score'] > games['team_a_score'], 1, 0)
games['label_X'] = np.where(games['team_h_score'] == games['team_a_score'], 1, 0)
games['label_2'] = np.where(games['team_h_score'] < games['team_a_score'], 1, 0)
games['train_score'] = np.where(games['finished'], 'train', 'score')

In [5]:
games_base = games[[
       # id
       'season_start_year', 
       'GW', 
       'id', 
       'team_h', 
       'team_a', 
       'train_score',

       # label
       'label_1', 
       'label_X', 
       'label_2', 

       # features
       'home',  
       'away', 
       'kickoff_year', 
       'kickoff_month', 
       'rounds_left']]

games_base.sort_values(['season_start_year', 'GW'])

,season_start_year,GW,id,team_h,team_a,train_score,label_1,label_X,label_2,home,away,kickoff_year,kickoff_month,rounds_left
380,19,1,1,10,14,train,1,0,0,LIV,NOR,2019,8,37
381,19,1,8,19,11,train,0,0,1,WHU,MCI,2019,8,37
382,19,1,2,3,15,train,0,1,0,BOU,SHU,2019,8,37
383,19,1,3,5,16,train,1,0,0,BUR,SOU,2019,8,37
384,19,1,4,7,8,train,0,1,0,CRY,EVE,2019,8,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,22,38,376,8,3,score,0,0,0,EVE,BOU,2023,5,0
376,22,38,377,11,18,score,0,0,0,LEE,TOT,2023,5,0
377,22,38,378,10,19,score,0,0,0,LEI,WHU,2023,5,0
378,22,38,379,14,9,score,0,0,0,MUN,FUL,2023,5,0


# Games features

In [17]:
games = pd.read_csv(os.getcwd() + "/artifacts/fetched_data/get_game_list.csv")

home = games.rename(columns = {'home':'team', 'team_h':'team_id_season'}).drop(['away', 'team_a'], axis=1)
home['home'] = 1

away = games.rename(columns = {'away':'team', 'team_a':'team_id_season'}).drop(['home', 'team_h'], axis=1)
away['home'] = 0

team_games = pd.concat([home, away])

team_games['win'] = np.where((team_games.home == 1) & (team_games['team_h_score'] > team_games['team_a_score']), 1, 
                              np.where((team_games.home == 0) & (team_games['team_h_score'] < team_games['team_a_score']), 1, 0))
team_games['draw'] = np.where((team_games['team_h_score'] == team_games['team_a_score']), 1, 0)
team_games['loss'] = np.where((team_games.home == 1) & (team_games['team_h_score'] < team_games['team_a_score']), 1, 
                              np.where((team_games.home == 0) & (team_games['team_h_score'] > team_games['team_a_score']), 1, 0))

team_games['goals_scored'] = np.where(team_games.home == 1, team_games['team_h_score'], team_games['team_a_score'])
team_games['goals_conceded'] = np.where(team_games.home == 1, team_games['team_a_score'], team_games['team_h_score'])

team_games['kickoff'] = pd.to_datetime(team_games['kickoff'])
team_games = team_games.sort_values('kickoff').reset_index(drop=True)

team_games['next_id'] = team_games.groupby('team_id_season')['id'].shift(-1)

team_games = team_games.drop(['team_h_score', 'team_a_score'], axis=1)


In [18]:
team_games[(team_games.team_id_season==1) & (team_games.season_start_year == 22)]

,GW,id,team,team_id_season,finished,kickoff,season_start_year,home,win,draw,loss,goals_scored,goals_conceded,next_id
2280,1,1,ARS,1,True,2022-08-05 19:00:00+00:00,22,0,1,0,0,2.0,0.0,11.0
2310,2,11,ARS,1,True,2022-08-13 14:00:00+00:00,22,1,1,0,0,4.0,2.0,21.0
2331,3,21,ARS,1,True,2022-08-20 16:30:00+00:00,22,0,1,0,0,3.0,0.0,31.0
2353,4,31,ARS,1,True,2022-08-27 16:30:00+00:00,22,1,1,0,0,2.0,1.0,41.0
2373,5,41,ARS,1,True,2022-08-31 18:30:00+00:00,22,1,1,0,0,2.0,1.0,56.0
2399,6,56,ARS,1,True,2022-09-04 15:30:00+00:00,22,0,0,0,1,1.0,3.0,72.0
2410,8,72,ARS,1,True,2022-09-18 11:00:00+00:00,22,0,1,0,0,3.0,0.0,81.0
2415,9,81,ARS,1,True,2022-10-01 11:30:00+00:00,22,1,1,0,0,3.0,1.0,91.0
2449,10,91,ARS,1,True,2022-10-09 15:30:00+00:00,22,1,1,0,0,3.0,2.0,104.0
2466,11,104,ARS,1,True,2022-10-16 13:00:00+00:00,22,0,1,0,0,1.0,0.0,127.0


## Overall form

In [19]:
team_games = team_games.sort_values(['season_start_year', 'team_id_season', 'kickoff']).reset_index(drop=True)

team_games['win_share_latest_5_games'] = team_games.groupby(['season_start_year', 'team_id_season'])['win'].rolling(window=5, min_periods=1).mean().reset_index(drop=True)
team_games['draw_share_latest_5_games'] = team_games.groupby(['season_start_year', 'team_id_season'])['draw'].rolling(window=5, min_periods=1).mean().reset_index(drop=True)
team_games['loss_share_latest_5_games'] = team_games.groupby(['season_start_year', 'team_id_season'])['loss'].rolling(window=5, min_periods=1).mean().reset_index(drop=True)

team_games['avg_goals_scored_latest_5_games'] = team_games.groupby(['season_start_year', 'team_id_season'])['goals_scored'].rolling(window=5, min_periods=1).mean().reset_index(drop=True)
team_games['avg_goals_conceded_latest_5_games'] = team_games.groupby(['season_start_year', 'team_id_season'])['goals_conceded'].rolling(window=5, min_periods=1).mean().reset_index(drop=True)


## Home form

In [46]:
home_games = team_games.loc[team_games.home == 1].sort_values(['season_start_year', 'team_id_season', 'kickoff']).reset_index(drop=True)
home_games['next_id_home'] = home_games.groupby('team_id_season')['id'].shift(-1)

home_games['win_share_latest_5_games_home'] = home_games.groupby(['season_start_year', 'team_id_season'])['win'].rolling(window=5, min_periods=1).mean().reset_index(drop=True)
home_games['draw_share_latest_5_games_home'] = home_games.groupby(['season_start_year', 'team_id_season'])['draw'].rolling(window=5, min_periods=1).mean().reset_index(drop=True)
home_games['loss_share_latest_5_games_home'] = home_games.groupby(['season_start_year', 'team_id_season'])['loss'].rolling(window=5, min_periods=1).mean().reset_index(drop=True)

home_games['avg_goals_scored_latest_5_games_home'] = home_games.groupby(['season_start_year', 'team_id_season'])['goals_scored'].rolling(window=5, min_periods=1).mean().reset_index(drop=True)
home_games['avg_goals_conceded_latest_5_games_home'] = home_games.groupby(['season_start_year', 'team_id_season'])['goals_conceded'].rolling(window=5, min_periods=1).mean().reset_index(drop=True)

home_games = home_games[['team_id_season', 
                         'next_id_home', 
                         'season_start_year', 
                         'win_share_latest_5_games_home', 
                         'draw_share_latest_5_games_home', 
                         'loss_share_latest_5_games_home', 
                         'avg_goals_scored_latest_5_games_home', 
                         'avg_goals_conceded_latest_5_games_home']]

## Away form

In [47]:
away_games = team_games.loc[team_games.home == 0].sort_values(['season_start_year', 'team_id_season', 'kickoff']).reset_index(drop=True)
away_games['next_id_away'] = away_games.groupby('team_id_season')['id'].shift(-1)

away_games['win_share_latest_5_games_away'] = away_games.groupby(['season_start_year', 'team_id_season'])['win'].rolling(window=5, min_periods=1).mean().reset_index(drop=True)
away_games['draw_share_latest_5_games_away'] = away_games.groupby(['season_start_year', 'team_id_season'])['draw'].rolling(window=5, min_periods=1).mean().reset_index(drop=True)
away_games['loss_share_latest_5_games_away'] = away_games.groupby(['season_start_year', 'team_id_season'])['loss'].rolling(window=5, min_periods=1).mean().reset_index(drop=True)

away_games['avg_goals_scored_latest_5_games_away'] = away_games.groupby(['season_start_year', 'team_id_season'])['goals_scored'].rolling(window=5, min_periods=1).mean().reset_index(drop=True)
away_games['avg_goals_conceded_latest_5_games_away'] = away_games.groupby(['season_start_year', 'team_id_season'])['goals_conceded'].rolling(window=5, min_periods=1).mean().reset_index(drop=True)

away_games = away_games[['team_id_season', 
                         'next_id_away', 
                         'season_start_year', 
                         'win_share_latest_5_games_away', 
                         'draw_share_latest_5_games_away', 
                         'loss_share_latest_5_games_away', 
                         'avg_goals_scored_latest_5_games_away', 
                         'avg_goals_conceded_latest_5_games_away']]

In [77]:
team_games_home = team_games.rename(columns={
                         'win_share_latest_5_games':'win_share_latest_5_games_home_overall', 
                         'draw_share_latest_5_games':'draw_share_latest_5_games_home_overall', 
                         'loss_share_latest_5_games':'loss_share_latest_5_games_home_overall', 
                         'avg_goals_scored_latest_5_games':'avg_goals_scored_latest_5_games_home_overall', 
                         'avg_goals_conceded_latest_5_games':'avg_goals_conceded_latest_5_games_home_overall'})

team_games_home = team_games_home[['team_id_season', 
                                    'next_id', 
                                    'season_start_year', 
                                    'win_share_latest_5_games_home_overall', 
                                    'draw_share_latest_5_games_home_overall', 
                                    'loss_share_latest_5_games_home_overall', 
                                    'avg_goals_scored_latest_5_games_home_overall', 
                                    'avg_goals_conceded_latest_5_games_home_overall']]

team_games_away = team_games.rename(columns={
                         'win_share_latest_5_games':'win_share_latest_5_games_away_overall', 
                         'draw_share_latest_5_games':'draw_share_latest_5_games_away_overall', 
                         'loss_share_latest_5_games':'loss_share_latest_5_games_away_overall', 
                         'avg_goals_scored_latest_5_games':'avg_goals_scored_latest_5_games_away_overall', 
                         'avg_goals_conceded_latest_5_games':'avg_goals_conceded_latest_5_games_away_overall'})

team_games_away = team_games_away[['team_id_season', 
                                    'next_id', 
                                    'season_start_year', 
                                    'win_share_latest_5_games_away_overall', 
                                    'draw_share_latest_5_games_away_overall', 
                                    'loss_share_latest_5_games_away_overall', 
                                    'avg_goals_scored_latest_5_games_away_overall', 
                                    'avg_goals_conceded_latest_5_games_away_overall']]

data = games_base.merge(team_games_home, left_on=['season_start_year', 'team_h', 'id'], right_on=['season_start_year', 'team_id_season', 'next_id'], how='inner').drop(['next_id', 'team_id_season'], axis=1)
data = data.merge(team_games_away, left_on=['season_start_year', 'team_a', 'id'], right_on=['season_start_year', 'team_id_season', 'next_id'], how='inner').drop(['next_id', 'team_id_season'], axis=1)
data = data.merge(home_games, left_on=['season_start_year', 'team_h', 'id'], right_on=['season_start_year', 'team_id_season', 'next_id_home'], how='inner').drop(['next_id_home', 'team_id_season'], axis=1)
data = data.merge(away_games, left_on=['season_start_year', 'team_a', 'id'], right_on=['season_start_year', 'team_id_season', 'next_id_away'], how='inner').drop(['next_id_away', 'team_id_season'], axis=1)

In [79]:
data

,season_start_year,GW,id,team_h,team_a,train_score,label_1,label_X,label_2,home,away,kickoff_year,kickoff_month,rounds_left,win_share_latest_5_games_home_overall,draw_share_latest_5_games_home_overall,loss_share_latest_5_games_home_overall,avg_goals_scored_latest_5_games_home_overall,avg_goals_conceded_latest_5_games_home_overall,win_share_latest_5_games_away_overall,draw_share_latest_5_games_away_overall,loss_share_latest_5_games_away_overall,avg_goals_scored_latest_5_games_away_overall,avg_goals_conceded_latest_5_games_away_overall,win_share_latest_5_games_home,draw_share_latest_5_games_home,loss_share_latest_5_games_home,avg_goals_scored_latest_5_games_home,avg_goals_conceded_latest_5_games_home,win_share_latest_5_games_away,draw_share_latest_5_games_away,loss_share_latest_5_games_away,avg_goals_scored_latest_5_games_away,avg_goals_conceded_latest_5_games_away
0,22,3,29,18,20,train,1,0,0,TOT,WOL,2022,8,35,0.5,0.5,0.0,3.0,1.5,0.0,0.5,0.5,0.5,1.0,1.0,0.0,0.0,4.0,1.0,0.0,0.0,1.0,1.0,2.0
1,22,3,22,7,2,train,1,0,0,CRY,AVL,2022,8,35,0.0,0.5,0.5,0.5,1.5,0.5,0.0,0.5,1.0,1.5,0.0,0.0,1.0,0.0,2.0,0.0,0.0,1.0,0.0,2.0
2,22,3,23,8,16,train,0,1,0,EVE,NFO,2022,8,35,0.0,0.0,1.0,0.5,1.5,0.5,0.0,0.5,0.5,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0
3,22,3,24,9,4,train,1,0,0,FUL,BRE,2022,8,35,0.0,1.0,0.0,1.0,1.0,0.5,0.5,0.0,3.0,1.0,0.0,1.0,0.0,2.0,2.0,0.0,1.0,0.0,2.0,2.0
4,22,3,26,10,17,train,0,0,1,LEI,SOU,2022,8,35,0.0,0.5,0.5,2.0,3.0,0.0,0.5,0.5,1.5,3.0,0.0,1.0,0.0,2.0,2.0,0.0,0.0,1.0,1.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1434,21,38,376,7,13,train,1,0,0,CRY,MUN,2022,5,0,0.4,0.4,0.2,1.2,1.0,0.2,0.2,0.6,1.0,2.4,0.4,0.6,0.0,1.0,0.2,0.0,0.0,1.0,0.4,3.2
1435,21,38,377,9,16,train,1,0,0,LEI,SOU,2022,5,0,0.4,0.2,0.4,2.2,1.4,0.0,0.2,0.8,0.8,2.2,0.6,0.2,0.2,1.6,0.8,0.0,0.4,0.6,0.6,2.4
1436,21,38,378,11,20,train,1,0,0,LIV,WOL,2022,5,0,0.8,0.2,0.0,1.6,0.6,0.0,0.4,0.6,0.8,2.4,0.8,0.2,0.0,2.0,0.2,0.2,0.2,0.6,0.6,1.0
1437,21,38,379,12,2,train,1,0,0,MCI,AVL,2022,5,0,0.8,0.2,0.0,4.2,0.8,0.4,0.4,0.2,1.6,1.0,0.8,0.2,0.0,3.8,0.8,0.4,0.2,0.4,1.6,1.0
